通用配置

In [1]:
# 读入配置文件
import yaml

with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

api_key = config['api_key']
base_url = config['base_url']
model = config['model']

# 定义一个工具
tools_description = [{
    "type": "function",
    "function": {
        "name": "python_repl",
        "description": "A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
        "parameters": {
            "type": "object",
            "properties": {
                "tool_input": {
                    "description": "Python executable code",
                    "type": "str"
                }
            },
            "required": [
                "tool_input"
            ]
        }
    }
}]

调用实验

In [2]:
# 使用openai sdk
from openai import OpenAI

client = OpenAI(
    api_key=api_key,
    base_url=base_url
)

user_prompt = "30以内质数有哪些"

completion = client.chat.completions.create(
    model=model,  
    messages=[      
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.1,
    tools=tools_description,
    tool_choice="auto"
) 

completion.choices[0].message.model_dump()

{'content': None,
 'refusal': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [{'id': 'call_20240903174401191b423d051f4be4',
   'function': {'arguments': '{"tool_input": "def is_prime(n):\\n    if n <= 1:\\n        return False\\n    for i in range(2, int(n**0.5) + 1):\\n        if n % i == 0:\\n            return False\\n    return True\\n\\nprimes = [n for n in range(2, 31) if is_prime(n)]\\nprint(primes)"}',
    'name': 'python_repl'},
   'type': 'function',
   'index': 0}]}

In [4]:
# Langchain SDK
from langchain_openai import ChatOpenAI
from langchain_experimental.tools.python.tool import PythonAstREPLTool

llm = ChatOpenAI(
    temperature=0.1,
    model=model,
    openai_api_key=api_key,
    openai_api_base=base_url
)

python_repl = PythonAstREPLTool()
tools = [python_repl]

llm_with_tools = llm.bind_tools(tools)

user_prompt = "编写python代码，根据执行结果回答30以内质数有哪些"

ai_msg = llm_with_tools.invoke(user_prompt)
ai_msg.tool_calls


[{'name': 'python_repl_ast',
  'args': {'query': 'def is_prime(n):\n    if n <= 1:\n        return False\n    for i in range(2, int(n**0.5) + 1):\n        if n % i == 0:\n            return False\n    return True\n\nprimes = [n for n in range(2, 31) if is_prime(n)]\nprimes'},
  'id': 'call_20240903174424b3048c4afbff481f',
  'type': 'tool_call'}]

In [5]:
# 查看langchain工具对角的tool_call_schema
python_repl.tool_call_schema.schema()

{'title': 'python_repl_ast',
 'description': 'A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.',
 'type': 'object',
 'properties': {'query': {'title': 'Query',
   'description': 'code snippet to run',
   'type': 'string'}},
 'required': ['query']}

react agent

In [6]:
# hwchase17/react - Langchain

from langchain_core.prompts import PromptTemplate

prompt = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt_template = PromptTemplate(template=prompt)

In [7]:
from langchain.agents import AgentExecutor, create_react_agent

agent = create_react_agent(llm, tools, prompt_template)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

user_prompt = "1+1=？"

result = agent_executor.invoke({"input":user_prompt})



> Entering new AgentExecutor chain...
The question is asking for the result of the simple arithmetic operation 1+1. This can be directly calculated.

Action: python_repl_ast
Action Input: 1 + 1
ObservationNameError: name 'Observation' is not definedIt seems there was an error in the format of my response. Let's correct that and try the calculation again.

Action: python_repl_ast
Action Input: 1 + 1
ObservationNameError: name 'Observation' is not definedIt appears there's a misunderstanding in the format I should use. I'll correct this and retry the calculation.

Action: python_repl_ast
Action Input: 1 + 1
ObservationNameError: name 'Observation' is not definedI apologize for the repeated mistake. Let's try this again with the correct format.

Action: python_repl_ast
Action Input: 1 + 1
ObservationNameError: name 'Observation' is not definedI apologize for the confusion earlier. Let's try to execute the calculation again with the correct format.

Action: python_repl_ast
Action Input: 

tool calling agent

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You are a helpful assistant'),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [17]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt_template)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

user_prompt = "当前系统时间是什么"

result = agent_executor.invoke({"input":user_prompt})



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'import datetime\ndatetime.datetime.now()'}`


2024-09-03 17:49:55.208622当前系统时间是2024年9月3日17点49分55秒。

> Finished chain.
